In [2]:
# comparitive models
import numpy as np # must use numpy ==1.19.5 and python=3.9
import pandas as pd
import geopandas as gpd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import(cross_validate, KFold, cross_val_score, RandomizedSearchCV)
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from mgwr.gwr import GWR
from mgwr.gwr import MGWR
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import json
import contextlib
import io

In [2]:
# reading in geodataframe from "DataPrep.ipynb"
dataframe = gpd.read_file('C:/Users/lewil/OneDrive - University of Bristol/Msc Geographic Data Science & Spatial Analytics/Dissertation/combined_df.gpkg')

dataframe.head()

,property_id,result_count,num_bathrooms,num_bedrooms,num_recepts,num_floors,price,last_marketed_days,marketed_duration,property_type_Bungalow,...,status_sale_under_offer,status_sold,IntRate,latitude,longitude,LAD11NM,CouncilTaxEst,POIdensity_score,distance_from_centre,geometry
0,2485433,2,1,2,1,0,260000,328,225,0.0,...,0.0,1.0,0.25,51.4301,0.0145,Lewisham,2288.65,6.0,72.687308,POINT (0.01450 51.43010)
1,2482388,1,1,3,2,0,350000,328,1,0.0,...,0.0,0.0,0.25,51.4250,0.0142,Bromley,2561.28,11.0,72.683921,POINT (0.01420 51.42500)
2,2483664,2,1,3,1,0,510000,342,297,0.0,...,0.0,1.0,0.10,51.4248,0.0240,Bromley,2561.28,0.0,72.676837,POINT (0.02400 51.42480)
3,2488692,2,2,3,2,0,475000,343,274,0.0,...,0.0,1.0,0.10,51.4271,-0.0048,Lewisham,2640.75,2.0,72.698864,POINT (-0.00480 51.42710)
4,2473496,3,2,2,1,0,375000,343,506,0.0,...,0.0,0.0,0.10,51.4091,0.0182,Bromley,2561.28,3.0,72.669866,POINT (0.01820 51.40910)


In [3]:
# subsampling df
df, _= train_test_split(dataframe, 
                             # train_size=0.005, 
                              random_state=0)

# # full dataset
# df = dataframe

In [4]:
df.columns

Index(['property_id', 'result_count', 'num_bathrooms', 'num_bedrooms',
       'num_recepts', 'num_floors', 'price', 'last_marketed_days',
       'marketed_duration', 'property_type_Bungalow', 'property_type_Cottage',
       'property_type_Detached bungalow', 'property_type_Detached house',
       'property_type_End terrace house', 'property_type_Flat',
       'property_type_Link-detached house', 'property_type_Maisonette',
       'property_type_Semi-detached bungalow',
       'property_type_Semi-detached house', 'property_type_Studio',
       'property_type_Terraced house', 'property_type_Town house',
       'property_type_other', 'status_for_sale', 'status_sale_under_offer',
       'status_sold', 'IntRate', 'latitude', 'longitude', 'LAD11NM',
       'CouncilTaxEst', 'POIdensity_score', 'distance_from_centre',
       'geometry'],
      dtype='object')

In [1]:
# making non-geographic X 
X = df.select_dtypes(include=['number']).drop(columns=['price', 
                                                       'property_id', 
                                                       'latitude',
                                                       'longitude',
                                                       'POIdensity_score',
                                                       'distance_from_centre',
                                                       'CouncilTaxEst'
                                                      ]).values
# setting y
y = df['price'].values

# checking sampled size
print(X.shape, y.shape)

NameError: name 'df' is not defined

In [6]:
# making XGB model without explicit geographic features for comparison
# setting hyperparamter tuning space (samller for personal computer)
search_spaces = {
    'n_estimators': [100, 500],
    'max_depth': [6, 10], # mazimum tree depth - deeper for more complex patterns but can lead to overfitting or finding
    'learning_rate': [0.01, 0.1], # shrinkage step sizes - lower values are more robust (less overfitting)
    'subsample': [0.4, 0.8], # percentage of rows used for each subsample
    'colsample_bytree': [0.4, 0.8], # percentage of columns ued for each subsample
    'gamma': [0, 0.5], # min loss reduction to make further partitions (leaves)
    'reg_alpha': [0.01, 1], # regularises weights
    'reg_lambda': [1, 2] # regularises weights
 }


# # setting hyperparamter tuning space (bigger for HPC)
# search_spaces = {
#     'n_estimators': [100, 250, 500, 750],
#     'max_depth': [4, 6, 8, 10, 12, 14], # mazimum tree depth - deeper for more complex patterns but can lead to overfitting or finding
#     'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.5], # shrinkage step sizes - lower values are more robust (less overfitting)
#     'subsample': [0.4, 0.6, 0.8, 1.0], # percentage of rows used for each subsample
#     'colsample_bytree': [0.4, 0.6, 0.8, 1.0], # percentage of columns ued for each subsample
#     'gamma': [0, 0.1, 0.25, 0.5], # min loss reduction to make further partitions (leaves)
#     'reg_alpha': [0, 0.01, 0.1, 1, 10], # regularises weights
#     'reg_lambda': [0.5, 1, 1.5, 2, 2.5, 3] # regularises weights
#  }


# using random search
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:absoluteerror', # reg:squarederror
                               random_state=0),
    param_distributions=search_spaces,
    n_iter=50, # the number of paramter setting sampled
    scoring= 'neg_mean_absolute_error', # more robust to outliers that 'neg_mean_squared_error'
    cv=5, # 5 fold cross validation
    n_jobs=-1, # ensures using maximum cores
    verbose=1, 
    random_state=0,
    error_score='raise' # raises error if fitting fails
)

try:
    random_search.fit(X, y)
except ValueError as e:
    print(e)


# Print the best parameters and best score
print("Best parameters:", random_search.best_params_)
print("Best score (neg absolute error):", random_search.best_score_)

In [7]:
# saving nongeo XGB results to .txt
# best_model = random_search.best_estimator_
# best_model.fit(X, y)

# calculating R2 score
y_pred = best_model.predict(X)
r2 = r2_score(y, y_pred)


# saving  results to .txt
filename = "nonGeoXGB_results.txt"

with open(filename, 'w') as f:
    f.write("RandomizedSearchCV Results\n")
    f.write("==========================\n\n")
    
    f.write("Best parameters:\n")
    f.write(json.dumps(random_search.best_params_, indent=4))  # Use json.dumps for pretty printing
    f.write("\n\n")
    
    f.write("Best score (neg absolute error):\n")
    f.write(f"{random_search.best_score_:.4f}\n")
    
    f.write("R2 Score:\n")
    f.write(f"{r2:.4f}\n")

print(f"\nResults saved to {filename}")

In [8]:
# resetting X to include geographic features
X = df.select_dtypes(include=['number']).drop(columns=['price', 'property_id']).values

# checking sampled size
print(X.shape, y.shape)

(988, 30) (988,)


In [9]:
# checking for NAs

# checking NAs in X
print(np.isnan(X).sum())

# checking for missing rows
print(np.isnan(X).any(axis=1).sum())

# checkign for NAs in y
print(np.isnan(y).sum())


X = X[~np.isnan(X).any(axis=1)]
y = y[~np.isnan(y)]

0


In [10]:
# Fitting GeoXGBoost model
# xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=0)

# setting hyperparamter tuning space (samller for personal comp)
search_spaces = {
    'n_estimators': [100, 500],
    'max_depth': [6, 10], # mazimum tree depth - deeper for more complex patterns but can lead to overfitting or finding
    'learning_rate': [0.01, 0.1], # shrinkage step sizes - lower values are more robust (less overfitting)
    'subsample': [0.4, 0.8], # percentage of rows used for each subsample
    'colsample_bytree': [0.4, 0.8], # percentage of columns ued for each subsample
    'gamma': [0, 0.5], # min loss reduction to make further partitions (leaves)
    'reg_alpha': [0.01, 1], # regularises weights
    'reg_lambda': [1, 2] # regularises weights
 }


# setting hyperparamter tuning space (bigger for HPC)
# search_spaces = {
#     'n_estimators': [100, 250, 500, 750],
#     'max_depth': [4, 6, 8, 10, 12, 14], # mazimum tree depth - deeper for more complex patterns but can lead to overfitting or finding
#     'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.5], # shrinkage step sizes - lower values are more robust (less overfitting)
#     'subsample': [0.4, 0.6, 0.8, 1.0], # percentage of rows used for each subsample
#     'colsample_bytree': [0.4, 0.6, 0.8, 1.0], # percentage of columns ued for each subsample
#     'gamma': [0, 0.1, 0.25, 0.5], # min loss reduction to make further partitions (leaves)
#     'reg_alpha': [0, 0.01, 0.1, 1, 10], # regularises weights
#     'reg_lambda': [0.5, 1, 1.5, 2, 2.5, 3] # regularises weights
#  }


# using random search
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:absoluteerror', # reg:squarederror
                               random_state=0),
    param_distributions=search_spaces,
    n_iter=50, # the number of paramter setting sampled
    scoring= 'neg_mean_absolute_error', # more robust to outliers than 'neg_mean_squared_error',
    cv=5, # 5 fold cross validation
    n_jobs=-1, # ensures using maximum cores
    verbose=1, 
    random_state=0,
    error_score='raise' # raises error if fitting fails
)

try:
    random_search.fit(X, y)
except ValueError as e:
    print(e)


# Print the best parameters and best score
print("Best parameters:", random_search.best_params_)
print("Best score (neg absolute error):", random_search.best_score_)


In [11]:
# calculating feature imprtances with permutations

# fitting best estimator on the entire dataset
best_model = random_search.best_estimator_
best_model.fit(X, y)

# calculating R2 score
y_pred = best_model.predict(X)
r2 = r2_score(y, y_pred)

# calculating permutation importances
perm_importance = permutation_importance(best_model, X, y, 
                                         n_repeats=50, 
                                         random_state=0)

# setting feature names 
feature_names = df.select_dtypes(include=['number']).drop(columns=['price', 'property_id']).columns.tolist()

# retrieving the importances
importances = perm_importance.importances_mean

# retrieving standard deviations
stds = perm_importance.importances_std

# printing results
for i in importances.argsort()[::-1]:
    print(f"{feature_names[i]}: {importances[i]:.4f} ± {stds[i]:.4f}")

# saving  results to .txt file
with open('XGB_feature_importances.txt', 'w') as file:
    for i in importances.argsort()[::-1]:
        file.write(f"{feature_names[i]}: {importances[i]:.4f} ± {stds[i]:.4f}\n")

In [12]:
# sorting importances for plotting
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]
sorted_stds = stds[indices]
sorted_feature_names = [feature_names[i] for i in indices]

# only showing most important in graph
n_top_params = 12
sorted_importances = sorted_importances[:n_top_params]
sorted_stds = sorted_stds[:n_top_params]
sorted_feature_names = sorted_feature_names[:n_top_params]


# setting seaborn style
sns.set(style="whitegrid")

# Plotting on a log scale with specific ticks and labels
label_font = 14
title_font = 16
annot_font = 12
xtick_font = 12

# making pattern for differentiation between geographic and non-geo (colourblind friendly this way)
def stripe_pattern(color1, color2, n_stripes=10):
    pattern = np.ones((n_stripes, n_stripes, 4))
    pattern[:,:,0] = color1[0]  # red 
    pattern[:,:,1] = color1[1]  # green 
    pattern[:,:,2] = color1[2]  # blue 
    pattern[:,:,3] = color1[3]  # alpha 
    
    # adding the diagonal stripes
    for i in range(n_stripes):
        pattern[i, i, :] = color2
        if i > 0:
            pattern[i-1, i, :] = color2
        if i < n_stripes-1:
            pattern[i+1, i, :] = color2
    
    return pattern

stripe_texture = stripe_pattern(plt.cm.colors.to_rgba('skyblue'), 
                                plt.cm.colors.to_rgba('#3cb44b'))

# defining geographic features for plotting
geographic_features = ["CouncilTaxEst", "longitude", "latitude", "distance_from_centre", "POIdensity_score"]


fig, ax = plt.subplots(figsize=(12, 8))

for i, (importance, std, feature) in enumerate(zip(sorted_importances, sorted_stds, sorted_feature_names)):
    if feature in geographic_features:
        rect = ax.bar(i, importance, yerr=std, color='skyblue', edgecolor='k', capsize=5)
        rect[0].set_hatch('///')  # This adds diagonal stripes
        rect[0].set_facecolor('skyblue')  # Set the base color
    else:
        ax.bar(i, importance, yerr=std, color='skyblue', edgecolor='k', capsize=5)

plt.xticks(range(len(sorted_feature_names)), sorted_feature_names, 
           rotation=45, 
           ha='right',
           fontsize=xtick_font)
plt.ylabel("Permutation of Importance", fontsize=label_font)
plt.xlabel("Features", fontsize=label_font)
plt.title("XGBoost Feature Permutation Importance (Log Scale)", fontsize=title_font)
plt.yscale('log') # log scale to see differences more clearly
ticks = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
plt.yticks(ticks, ticks)

# adding annotations
for i, v in enumerate(sorted_importances):
    plt.text(i, v + 0.01, f"{v:.3f}", 
             ha='center', 
             va='bottom', 
             fontsize=annot_font)

sns.despine(left=True)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# adding legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='skyblue', edgecolor='k', hatch='///', label='Geographic'),
                   Patch(facecolor='skyblue', edgecolor='k', label='Non-Geographic')]
plt.legend(handles=legend_elements, loc='upper right')

# saving to .png
plt.savefig('XGB_pi_fig.png', dpi=400)
plt.show()

In [13]:
# saving XGB results to .txt

# Save the results to a text file
filename = "XGB_results.txt"

with open(filename, 'w') as f:
    f.write("RandomizedSearchCV Results\n")
    f.write("==========================\n\n")
    
    f.write("Best parameters:\n")
    f.write(json.dumps(random_search.best_params_, indent=4))  # Use json.dumps for pretty printing
    f.write("\n\n")
    
    f.write("Best score (neg absolute error):\n")
    f.write(f"{random_search.best_score_:.4f}\n")
    
    f.write("R2 Score:\n")
    f.write(f"{r2:.4f}\n")

print(f"\nResults saved to {filename}")

In [14]:
# before running GWR model looking for correlation in variables
correlation_matrix = df.corr()
#print(correlation_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [15]:
# looking for vairbales with low varience
print(df.var())

In [16]:
# Defining predict function for GWR
def predict(model, coords, X, y, coords_new, X_new, ridge_lambda=1.0):
    # checking dimensions
    print("Starting prediction...")
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")
    print(f"Shape of coords: {coords.shape}")
    print(f"Shape of X_new: {X_new.shape}")
    print(f"Shape of coords_new: {coords_new.shape}")
    
    # num training points
    n = X.shape[0]
    # num prediction points
    m = X_new.shape[0]
    
    # calculating the squared Euclidean distances
    dists = np.sum((coords[:, np.newaxis, :] - coords_new[np.newaxis, :, :])**2, axis=2)
    
    # retrieving bandwidths (if not scalar then take first from array)
    bw = model.bw if isinstance(model.bw, float) else model.bw[0]
    
    # calculating weights
    w = np.exp(-(dists / (bw))**2)
    
    # normalising weights
    w_sum = w.sum(axis=0)
    w = w / w_sum
    
    # creating array to store predictions
    predictions = np.zeros(m)
    
    # calcualting predictions
    for i in range(m):
        wi = w[:, i]   # specific weight
        XtW = X.T * wi # calcuating with weight
        XtWX = XtW @ X # weighted sum of squares
        XtWy = XtW @ y # weighted sum of X time y
        
        # adding ridge penalty to handle multicollinearity
        XtWX_ridge = XtWX + ridge_lambda * np.eye(X.shape[1])
        
        # using above to find coefficients beta
        try:
            beta = np.linalg.solve(XtWX_ridge, XtWy).squeeze()
        except np.linalg.LinAlgError:
            # if matrix is singular then psuedo inverse
            print(f"Warning: Singular matrix for point {i}. Using pseudo-inverse.")
            beta = np.linalg.pinv(XtWX_ridge) @ XtWy.squeeze()
        # store prediction
        predictions[i] = X_new[i] @ beta
    
    print(f"Shape of predictions: {predictions.shape}")
    return predictions

In [17]:
# gwr
# Standardising X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape y if necessary
y_gwr = y.reshape((-1, 1))

# Retrieve coords
coords = df[['longitude', 'latitude']].values

# splitting  data into train and test sets (80:20)
X_train, X_test, y_train, y_test, coords_train, coords_test = train_test_split(
    X_scaled, y_gwr, coords, test_size=0.2, random_state=0)

# checking shapes
print("Train shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("coords_train shape:", coords_train.shape)
print("\nTest shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("coords_test shape:", coords_test.shape)

# slelecting bandwidth with BIC
try:
    bw = Sel_BW(coords_train, y_train, X_train,
                kernel='gaussian', fixed=True).search(criterion='BIC')
    print(f"Selected bandwidth: {bw}")
except np.linalg.LinAlgError:
    # if error then using default bandwidth from median
    print("Error in bandwidth selection. Using default bandwidth.")
    bw = np.median(np.sum(coords_train**2, axis=1)**0.5)
    print(f"Default bandwidth: {bw}")

# Fitting GWR model on training data with error handling
try:
    gwr_model = GWR(coords_train, y_train, X_train,
                    bw=bw, fixed=True, kernel='gaussian')
    gwr_results = gwr_model.fit()
    print("GWR model fitted successfully")
except np.linalg.LinAlgError as e:
    print(f"Error in GWR fitting: {str(e)}")
    exit()

#  out-of-sample prediction now
try:
    y_pred = predict(gwr_model, coords_train, X_train, y_train, coords_test, X_test)

    # calculating MSE and R2 on test set
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred.reshape(-1, 1))

    # printing results
    print("\nOut-of-sample Results:")
    print(f"MSE: {mse:.4f}")
    print(f"R-squared: {r2:.4f}")

    # printing model summary on training data
    print("\nModel Summary (Training Data):")
    gwr_results.summary()
    
except Exception as e:
    print(f"Error in prediction: {str(e)}")
    import traceback
    traceback.print_exc()

Train shapes:
X_train shape: (790, 30)
y_train shape: (790, 1)
coords_train shape: (790, 2)

Test shapes:
X_test shape: (198, 30)
y_test shape: (198, 1)
coords_test shape: (198, 2)
Selected bandwidth: 1.3
GWR model fitted successfully
Starting prediction...
Shape of X: (790, 30)
Shape of y: (790, 1)
Shape of coords: (790, 2)
Shape of X_new: (198, 30)
Shape of coords_new: (198, 2)
Shape of predictions: (198,)

Out-of-sample Results:
MSE: 279801590787.4448
R-squared: -1.1380

Model Summary (Training Data):
Model type                                                         Gaussian
Number of observations:                                                 790
Number of covariates:                                                    30

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       14219786111565.658
Log-likelihood:                                                  -10448.342
A

C:\Anaconda\lib\site-packages\spglm\base.py:247: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))


In [22]:
# saving gwr results to .txt file

# Perform out-of-sample prediction
try:
    y_pred = predict(gwr_model, coords_train, X_train, y_train, coords_test, X_test)

    # Calculate MSE, MAE, and R-squared
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred.reshape(-1, 1))

    # Print results
    print("\nOut-of-sample Results:")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R-squared: {r2:.4f}")
    
    gwr_results_summary = gwr_results.summary()

    # Save results to a text file
    filename = "gwr_results.txt"
    
    with open(filename, 'w') as f:
        f.write("GWR Model Results\n")
        f.write("=================\n\n")
        
        f.write("Data Shapes:\n")
        f.write(f"X_train shape: {X_train.shape}\n")
        f.write(f"y_train shape: {y_train.shape}\n")
        f.write(f"coords_train shape: {coords_train.shape}\n")
        f.write(f"X_test shape: {X_test.shape}\n")
        f.write(f"y_test shape: {y_test.shape}\n")
        f.write(f"coords_test shape: {coords_test.shape}\n\n")
        
        f.write(f"Selected bandwidth: {bw}\n\n")
        
        f.write("Out-of-sample Results:\n")
        f.write(f"MSE: {mse:.4f}\n")
        f.write(f"MAE: {mae:.4f}\n")
        f.write(f"R-squared: {r2:.4f}\n\n")
        
        f.write("Model Summary (Training Data):\n")
        
        # must be redirected by contextlib to save .summary to .txt
        summary_str = io.StringIO()
        with contextlib.redirect_stdout(summary_str):
            gwr_results.summary()
        f.write(summary_str.getvalue())
        
    print(f"\nResults saved to {filename}")

except Exception as e:
    print(f"Error in prediction: {str(e)}")
    import traceback # only importing if needed
    traceback.print_exc()

Starting prediction...
Shape of X: (790, 30)
Shape of y: (790, 1)
Shape of coords: (790, 2)
Shape of X_new: (198, 30)
Shape of coords_new: (198, 2)
Shape of predictions: (198,)

Out-of-sample Results:
MSE: 279801590787.4448
MAE: 457134.3448
R-squared: -1.1380
Model type                                                         Gaussian
Number of observations:                                                 790
Number of covariates:                                                    30

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       14219786111565.658
Log-likelihood:                                                  -10448.342
AIC:                                                              20956.685
AICc:                                                             20957.002
BIC:                                                           14219786106494.914
R2:              

C:\Anaconda\lib\site-packages\spglm\base.py:247: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
